In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns', 50)

In [2]:
data = pd.read_csv('heart_num_0228.csv')
Q1 = data[['SleepHours', 'BMI']].quantile(q = 0.25)
Q3 = data[['SleepHours', 'BMI']].quantile(q = 0.75)
IQR = Q3-Q1
data_iqr = data[(data['SleepHours'] <= Q3['SleepHours']+IQR['SleepHours']*1.5)&(data['SleepHours'] >= Q1['SleepHours']-IQR['SleepHours']*1.5)]
data_iqr = data_iqr[(data_iqr['BMI'] <= Q3['BMI']+IQR['BMI']*1.5)&(data_iqr['BMI'] >= Q1['BMI']-IQR['BMI']*1.5)]
print('Original data:', len(data))
print('Remove Outlier data:', len(data_iqr))

Original data: 274034
Remove Outlier data: 261812


In [3]:
X = data_iqr.drop('HadHeartAttack', axis = 1)
y = data_iqr['HadHeartAttack']

In [5]:
from imblearn.over_sampling import SMOTE

In [6]:
smo = SMOTE(random_state = 1234)
X_re, y_re = smo.fit_resample(X, y)
print('Original Data:', len(X))
print('Resampled Data:', len(X_re))

Original Data: 261812
Resampled Data: 494774


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_re, y_re, test_size = 0.2, random_state = 1234)

In [10]:
from sklearn.preprocessing import RobustScaler

In [11]:
rbs = RobustScaler()
X_trn = rbs.fit_transform(X_train)
X_tst = rbs.transform(X_test)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [13]:
print(X_trn.shape)

(395819, 46)


In [23]:
es = EarlyStopping(monitor = 'accuracy')

In [24]:
model = Sequential()
model.add(Dense(50, input_dim = 46, activation = 'relu'))
model.add(Dense(28, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [31]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [32]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', f1_m, precision_m, recall_m])
history = model.fit(X_trn, y_train, epochs = 100, batch_size = 10, callbacks = [es])
print('\n Accraucy: %.2f' %(model.evaluate(X_trn, y_train)))

Epoch 1/100
39582/39582 [==============================] - 74s 2ms/step - loss: 0.1500 - accuracy: 0.9388 - f1_m: 0.9294 - precision_m: 0.9561 - recall_m: 0.9182
Epoch 2/100
39582/39582 [==============================] - 105s 3ms/step - loss: 0.1457 - accuracy: 0.9407 - f1_m: 0.9315 - precision_m: 0.9579 - recall_m: 0.9200
Epoch 3/100
39582/39582 [==============================] - 111s 3ms/step - loss: 0.1423 - accuracy: 0.9426 - f1_m: 0.9330 - precision_m: 0.9595 - recall_m: 0.9211
Epoch 4/100
39582/39582 [==============================] - 133s 3ms/step - loss: 0.1392 - accuracy: 0.9438 - f1_m: 0.9347 - precision_m: 0.9602 - recall_m: 0.9235
Epoch 5/100
39582/39582 [==============================] - 124s 3ms/step - loss: 0.1367 - accuracy: 0.9448 - f1_m: 0.9360 - precision_m: 0.9609 - recall_m: 0.9252
Epoch 6/100
39582/39582 [==============================] - 112s 3ms/step - loss: 0.1341 - accuracy: 0.9463 - f1_m: 0.9375 - precision_m: 0.9619 - recall_m: 0.9269
Epoch 7/100
39582/39582

TypeError: must be real number, not list

In [36]:
loss, acc, f1, precision, recall = model.evaluate(X_trn, y_train)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {acc*100:.2f}%')
print(f'F1 Score: {f1:.4f}')

12370/12370 [==============================] - 15s 1ms/step - loss: 0.1261 - accuracy: 0.9497 - f1_m: 0.9466 - precision_m: 0.9731 - recall_m: 0.9247
Loss: 0.1261
Accuracy: 94.97%
F1 Score: 0.9466
